<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEmointGen_Add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
from nltk.corpus import sentiwordnet as swn
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Data/Tweets_clean_data.csv')
df = df.sample(frac=1,random_state=32)

Mounted at /content/drive
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...

In [2]:
category_dict = {"anger":[1,0,0,0],"sadness":[0,1,0,0],"fear":[0,0,1,0],"joy":[0,0,0,1]}
data_tweet = [x.lower().split() for x in df['Tweet']]
data_cat = np.array([category_dict[x] for x in df['Emotion']])

In [3]:
print(data_tweet[:5])
print(data_cat[:5])

[['with', 'their', 'faggy', 'colors', 'are', 'nice', 'is', 'ok', 'too', 'even', 'tho', 'some', 'might', 'take', 'offense', 'because', 'words', 'lol'], ['unbelievable', 'takes', '10', 'minutes', 'to', 'get', 'through', 'to', 'then', 'there', 'is', 'a', 'fault', 'and', 'the', 'call', 'hangs', 'up', 'treatcustomersfairly'], ['well', 'i', 'did', 'hear', 'once', 'before', 'that', 'girls', 'are', 'attracted', 'to', 'men', 'that', 'lok', 'like', 'their', 'dad', 'ok', 'hand'], ['agreed', 'so', 'tired', 'of', 'this', 'nonsense', 'soros', 'must', 'be', 'elated'], ['by', 'the', 'way', 'i', 'am', 'wearing', 'the', 'smile', 'you', 'gave', 'me', 'today', 'n', 'you', 'me']]
[[1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]]


In [4]:
# Parameters
Min_count = 0
Embedding_size = 200
Window_size = 5
Negative_sampling = 00

In [5]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [6]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [7]:
batch_size = 100
Max_input_size = max([len(x) for x in data_tweet])
data_tweet = [nltk.pos_tag(x) for x in data_tweet]
def getsent(word,tag):
  res=0
  try:
    x = swn.senti_synset(word+'.'+tag[0].lower()+'.01')
    res =  (x.pos_score()-x.neg_score())
  finally:
    return res 
data_tweet = [[(i[0],i[1],getsent(i[0],i[1])) for i in x] for x in data_tweet]

In [8]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.word_vec(i[0]),i[1],i[2]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data_tweet, data_cat, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
result_table = [0,0,0,0,0,0,0,0,0]

In [10]:
epochs = 60

In [11]:
from keras import backend as K
def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recalls = true_positives / (possible_positives + K.epsilon())
        return recalls
def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precisions = true_positives / (predicted_positives + K.epsilon())
        return precisions
def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [12]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(64))
model_sg.add(tf.keras.layers.Dropout(0.2))
model_sg.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg.add(tf.keras.layers.Dropout(0.2))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
sg_train_gen=vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size)
sg_val_gen=vec_gen(w2v_sg,vocab_sg,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_sg_lstm = model_sg.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 14s 140ms/step - loss: 1.3822 - accuracy: 0.3057 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3742 - val_accuracy: 0.3300 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 5s 118ms/step - loss: 1.3787 - accuracy: 0.3054 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3688 - val_accuracy: 0.3367 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/60
46/46 [==============================] - 5s 119ms/step - loss: 1.3771 - accuracy: 0.3106 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3703 - val_accuracy: 0.3333 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/60
46/46 [==============================] - 5s 120ms/step - loss: 1.3755 - accuracy: 0.3164 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - v

In [13]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_sg_bi = model_sg_bi.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 10s 146ms/step - loss: 1.3810 - accuracy: 0.2992 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3668 - val_accuracy: 0.3292 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 6s 127ms/step - loss: 1.3645 - accuracy: 0.3215 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3480 - val_accuracy: 0.3475 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/60
46/46 [==============================] - 6s 126ms/step - loss: 1.3185 - accuracy: 0.3794 - f1: 0.0394 - precision: 0.3362 - recall: 0.0240 - val_loss: 1.3274 - val_accuracy: 0.3633 - val_f1: 0.0494 - val_precision: 0.6236 - val_recall: 0.0258
Epoch 4/60
46/46 [==============================] - 6s 126ms/step - loss: 1.2599 - accuracy: 0.4282 - f1: 0.1674 - precision: 0.6796 - recall: 0.0977 - val_loss: 1.3091 - val_accuracy: 0.38

In [14]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(64))
model_sg_rnn.add(tf.keras.layers.Dropout(0.2))
model_sg_rnn.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg_rnn.add(tf.keras.layers.Dropout(0.2))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_sg_rnn = model_sg_rnn.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)


result_table[2]=history[1]
print("SG_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 9s 160ms/step - loss: 1.3879 - accuracy: 0.2668 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3695 - val_accuracy: 0.3300 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 7s 153ms/step - loss: 1.3773 - accuracy: 0.2978 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3691 - val_accuracy: 0.3300 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/60
46/46 [==============================] - 7s 151ms/step - loss: 1.3755 - accuracy: 0.3009 - f1: 2.1122e-04 - precision: 0.0036 - recall: 1.0878e-04 - val_loss: 1.3677 - val_accuracy: 0.3408 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/60
46/46 [==============================] - 7s 151ms/step - loss: 1.3738 - accuracy: 0.3040 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_lo

In [15]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(64))
model_cbow.add(tf.keras.layers.Dropout(0.2))
model_cbow.add(tf.keras.layers.Dense(64,activation='relu'))
model_cbow.add(tf.keras.layers.Dropout(0.2))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
cbow_train_gen=vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen(w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_cbow_lstm = model_cbow.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 8s 141ms/step - loss: 1.3821 - accuracy: 0.3101 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3727 - val_accuracy: 0.3300 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 5s 120ms/step - loss: 1.3783 - accuracy: 0.3054 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3700 - val_accuracy: 0.3367 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/60
46/46 [==============================] - 5s 119ms/step - loss: 1.3772 - accuracy: 0.3105 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3699 - val_accuracy: 0.3333 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/60
46/46 [==============================] - 5s 120ms/step - loss: 1.3758 - accuracy: 0.3164 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - va

In [16]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_cbow_bi = model_cbow_bi.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 10s 146ms/step - loss: 1.3821 - accuracy: 0.2929 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3655 - val_accuracy: 0.3292 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 6s 124ms/step - loss: 1.3630 - accuracy: 0.3222 - f1: 2.7681e-05 - precision: 0.0014 - recall: 1.3979e-05 - val_loss: 1.3481 - val_accuracy: 0.3650 - val_f1: 0.0017 - val_precision: 0.0833 - val_recall: 8.3333e-04
Epoch 3/60
46/46 [==============================] - 6s 125ms/step - loss: 1.3208 - accuracy: 0.3803 - f1: 0.0158 - precision: 0.2703 - recall: 0.0083 - val_loss: 1.3267 - val_accuracy: 0.3775 - val_f1: 0.0884 - val_precision: 0.5412 - val_recall: 0.0483
Epoch 4/60
46/46 [==============================] - 6s 126ms/step - loss: 1.2509 - accuracy: 0.4380 - f1: 0.1585 - precision: 0.6974 - recall: 0.0908 - val_loss: 1.2963 - val_accuracy: 0.4067 - val_f1:

In [17]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(64))
model_cbow_rnn.add(tf.keras.layers.Dropout(0.2))
model_cbow_rnn.add(tf.keras.layers.Dense(64,activation='relu'))
model_cbow_rnn.add(tf.keras.layers.Dropout(0.2))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_cbow_rnn = model_cbow_rnn.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 9s 161ms/step - loss: 1.3850 - accuracy: 0.2787 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3722 - val_accuracy: 0.3308 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 7s 148ms/step - loss: 1.3811 - accuracy: 0.2977 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3716 - val_accuracy: 0.3300 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/60
46/46 [==============================] - 7s 150ms/step - loss: 1.3772 - accuracy: 0.2984 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3810 - val_accuracy: 0.3200 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/60
46/46 [==============================] - 7s 151ms/step - loss: 1.3675 - accuracy: 0.3181 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - va

In [18]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove.add(tf.keras.layers.LSTM(64))
model_glove.add(tf.keras.layers.Dropout(0.2))
model_glove.add(tf.keras.layers.Dense(64,activation='relu'))
model_glove.add(tf.keras.layers.Dropout(0.2))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
glove_train_gen=glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
glove_val_gen=glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_glove_lstm = model_glove.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 4s 40ms/step - loss: 1.3782 - accuracy: 0.3092 - f1: 0.0055 - precision: 0.0251 - recall: 0.0031 - val_loss: 1.2567 - val_accuracy: 0.4350 - val_f1: 0.0017 - val_precision: 0.0833 - val_recall: 8.3333e-04
Epoch 2/60
46/46 [==============================] - 1s 29ms/step - loss: 1.2324 - accuracy: 0.4246 - f1: 0.1676 - precision: 0.4961 - recall: 0.1049 - val_loss: 1.1843 - val_accuracy: 0.4692 - val_f1: 0.2105 - val_precision: 0.7029 - val_recall: 0.1242
Epoch 3/60
46/46 [==============================] - 1s 29ms/step - loss: 1.1531 - accuracy: 0.4627 - f1: 0.3165 - precision: 0.6567 - recall: 0.2123 - val_loss: 1.1660 - val_accuracy: 0.4442 - val_f1: 0.3060 - val_precision: 0.6921 - val_recall: 0.1975
Epoch 4/60
46/46 [==============================] - 1s 28ms/step - loss: 1.1207 - accuracy: 0.4796 - f1: 0.3373 - precision: 0.6793 - recall: 0.2303 - val_loss: 1.1591 - val_accuracy: 0.4458 - val_f1: 0.2729 - val_precision: 0.7464 - val

In [19]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_glove_bi = model_glove_bi.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 6s 65ms/step - loss: 1.3795 - accuracy: 0.2991 - f1: 2.7681e-05 - precision: 0.0014 - recall: 1.3979e-05 - val_loss: 1.3126 - val_accuracy: 0.3992 - val_f1: 0.0017 - val_precision: 0.0833 - val_recall: 8.3333e-04
Epoch 2/60
46/46 [==============================] - 2s 43ms/step - loss: 1.2750 - accuracy: 0.4161 - f1: 0.0704 - precision: 0.5136 - recall: 0.0403 - val_loss: 1.1612 - val_accuracy: 0.5000 - val_f1: 0.2600 - val_precision: 0.6537 - val_recall: 0.1633
Epoch 3/60
46/46 [==============================] - 2s 43ms/step - loss: 1.0875 - accuracy: 0.5579 - f1: 0.3803 - precision: 0.6855 - recall: 0.2712 - val_loss: 0.9972 - val_accuracy: 0.5950 - val_f1: 0.5092 - val_precision: 0.6623 - val_recall: 0.4142
Epoch 4/60
46/46 [==============================] - 2s 42ms/step - loss: 0.9103 - accuracy: 0.6392 - f1: 0.5781 - precision: 0.7188 - recall: 0.4850 - val_loss: 0.9198 - val_accuracy: 0.6175 - val_f1: 0.5880 - val_precision: 0.68

In [20]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(64))
model_glove_rnn.add(tf.keras.layers.Dropout(0.2))
model_glove_rnn.add(tf.keras.layers.Dense(64,activation='relu'))
model_glove_rnn.add(tf.keras.layers.Dropout(0.2))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_glove_rnn = model_glove_rnn.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/60


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


46/46 [==============================] - 6s 96ms/step - loss: 1.3845 - accuracy: 0.2921 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3728 - val_accuracy: 0.3317 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 4s 80ms/step - loss: 1.3776 - accuracy: 0.2919 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3569 - val_accuracy: 0.3625 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/60
46/46 [==============================] - 4s 77ms/step - loss: 1.3643 - accuracy: 0.3274 - f1: 0.0083 - precision: 0.2149 - recall: 0.0043 - val_loss: 1.3599 - val_accuracy: 0.3383 - val_f1: 0.1115 - val_precision: 0.5290 - val_recall: 0.0625
Epoch 4/60
46/46 [==============================] - 4s 81ms/step - loss: 1.3410 - accuracy: 0.3512 - f1: 0.0880 - precision: 0.5583 - recall: 0.0493 - val_loss: 1.3546 - val_accuracy: 0.3658 - val_f1: 0.2

In [21]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])
# print(history_sg_bi.history)
# plt.title("Skip-Gram + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_sg_bi.history['loss'],c='b',label='loss')
# plt.plot(history_sg_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()
# plt.title("GloVe + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_glove_bi.history['loss'],c='b',label='loss')
# plt.plot(history_glove_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()

	Lstm			BiLstm			Rnn
sg [0.46000000834465027, 0.5699999928474426, 0.2914285659790039]
cbow [0.46785715222358704, 0.5492857098579407, 0.3149999976158142]
glove [0.772857129573822, 0.7685714364051819, 0.4242857098579407]
